In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import itertools
import json
import urllib

In [ ]:
def str2time(string, time_format="%Y-%m-%d %H:%M"):
    output = datetime.datetime.strptime(string, time_format)
    return time.mktime(output.timetuple())

In [ ]:
"""
play around with datetime objects

str2time("2015-10-17 14:50")
datetime.timedelta(hours = 1, minutes=20) + \
datetime.datetime(year=2015,month=10,day=17)
"""

In [76]:
# make a kino program to test algorithm on

the_martian = \
initialize_movie_df('the_martian',\
                    ['13:30', '16:50', '19:30', '23:10'],\
                    duration=144)

sicario = initialize_movie_df('sicario', \
                              ['20:00', '23:10'], \
                              duration=121)

alles_steht_kopf = initialize_movie_df('alles_steht_kopf', \
                                       ['14:20', '17:00'],\
                                       95)

black_mass = initialize_movie_df('black_mass',\
                                ['19:50', '23:00'],\
                                123)

hotel_transylvania = \
initialize_movie_df('hotel_transylvania', \
                    ['14:00', '16:40'],\
                   89)

maze_runner = initialize_movie_df('maze_runner',\
                                 ['13:50'],\
                                 130)

the_intern = initialize_movie_df('the_intern',\
                                ['17:00'],\
                                121)

regression = initialize_movie_df('regression',\
                                ['22:30'],
                                107)

pan = initialize_movie_df('pan',\
                          ['14:40'],\
                          111)

everest = initialize_movie_df('everest',\
                             ['22:00'],\
                             122)

kino_program = the_martian.\
append(black_mass, ignore_index=True).\
append(hotel_transylvania, ignore_index=True).\
append(maze_runner, ignore_index=True).\
append(the_intern, ignore_index=True).\
append(regression, ignore_index=True).\
append(pan, ignore_index=True).\
append(everest, ignore_index=True).\
append(sicario, ignore_index=True).\
append(alles_steht_kopf, ignore_index=True)

kino_program.sort('start_times',\
                  ascending=True,\
                 inplace=True)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [116]:
def find_next_showings(movie, kino_program, max_waiting_mins, acceptable_overlap_mins=0):
    
    # I anticipate 'movie' to be a row from a DataFrame, so we should
    # unpack elements using `.` and `iloc`...
    movie_title = movie.title.iloc[0]
    movie_end_time = movie.movie_end_time.iloc[0]
    
    latest_start_time = movie_end_time + datetime.timedelta(minutes=max_waiting_mins)
    overlap_duration = datetime.timedelta(minutes=acceptable_overlap_mins)
        
    # filter out:
    # 1. all future showings of the 'movie_title'
    # 2. all movies that start outside of the acceptable waiting period
    # include:
    # 1. movies that start during or after overlap period
    row_filter = \
        (kino_program.title != movie_title) & \
        (kino_program.movie_start_time >= movie_end_time - overlap_duration) & \
        (kino_program.movie_start_time <= latest_start_time)
        
    return kino_program[row_filter]

def movie_to_string(movie):
    """ 
    output looks something like:
    
    Alles steht Kopf (OV) (14:20-15:55)
    
    movie_title + ( + start_time + - + end_time + )
    
    movie should be a row from DataFrame with columns
    'title', 'start_time', and 'end_time'
    """
    return movie.title.iloc[0] + " (" + \
    movie.start_time.iloc[0].strftime("%H:%M") + "-" + \
    movie.movie_end_time.iloc[0].strftime("%H:%M") + ")"
    
def double_dip(starting_show, showtimes, max_waiting_mins=45, acceptable_overlap_mins=0):
    # filter out all future showings of 'starting_show':
    # this has to be done on 'showtimes' before any recursion
    # so the filtration can propagate forwards
    showtimes = showtimes[showtimes.title != starting_show.title.iloc[0]]
    
    # determine the next closest 'cluster' of shows
    next_showtimes = find_next_showings(starting_show,showtimes, max_waiting_mins, acceptable_overlap_mins)
    
    starting_show_info = movie_to_string(starting_show)
    
    # if there are no future shows, we're at a leaf node;
    # so, we should just output the show info
    if next_showtimes.empty:
        return starting_show_info
    
    # otherwise, for every show identified from 'find_next_shows'
    # we should collect the returned double-dips, "paths"
    paths = []
    
    for row in xrange(next_showtimes.shape[0]):
        next_show = next_showtimes.iloc[[row]]
        valid_double_dips = double_dip(next_show, showtimes, max_waiting_mins, acceptable_overlap_mins)
        
        """
        valid_double_dips can have two outputs:
         
        1. base-case: which is just movie info
        2. recursive-case: a list of valid (formatted) double dips
        
        In 1., we need to format the output to the form
        
            movie1_info -> movie2_info
        
        In 2., we just need to pre-pend starting_show_info to the
        already formatted output.
        """ 
        if isinstance(valid_double_dips, basestring):
            paths.append(starting_show_info + " -> " + valid_double_dips)
        else:
            # if valid_double_dips is a list (of lists) flatten it back
            # down to a list of strings
            if isinstance(valid_double_dips[0], list):
                valid_double_dips = [x for x in itertools.chain.from_iterable(valid_double_dips)]
            
            # prepend info
            tmp = [starting_show_info + " -> " + double_dips for double_dips in valid_double_dips]
            [paths.append(x) for x in tmp]
    
    return paths

def find_all_double_dips(showtimes, max_waiting_mins=45, acceptable_overlap_mins=0, movie_filter=''):
    double_dips = []
    for row in xrange(showtimes.shape[0]):
        tmp = double_dip(showtimes.iloc[[row]], showtimes, max_waiting_mins, acceptable_overlap_mins)
        if isinstance(tmp, list):
            [double_dips.append(x) for x in tmp]
        else: 
            double_dips.append(tmp)
    
    return double_dips

In [117]:
find_all_double_dips(kino_program)

[u'Der Marsianer - Rettet Mark Watney (OV) (13:30-16:14)',
 u'Der Marsianer - Rettet Mark Watney (OV) (16:50-19:34) -> Picknick mit B\xe4ren (OV) (19:40-21:44)',
 u'Der Marsianer - Rettet Mark Watney (OV) (16:50-19:34) -> Hotel Transsilvanien 2 (OV) (19:30-21:20)',
 u'Der Marsianer - Rettet Mark Watney (OV) (19:30-22:14)',
 u'Black Mass (OV) (20:15-22:38)',
 u'American Ultra (OV) (20:15-22:11)',
 u'Sicario (OV) (20:00-22:22)',
 u'Alles steht Kopf (OV) (14:20-16:15) -> Hotel Transsilvanien 2 (OV) (16:40-18:30)',
 u'Alles steht Kopf (OV) (15:00-16:55) -> Der Marsianer - Rettet Mark Watney (OV) (16:50-19:34) -> Picknick mit B\xe4ren (OV) (19:40-21:44)',
 u'Alles steht Kopf (OV) (15:00-16:55) -> Der Marsianer - Rettet Mark Watney (OV) (16:50-19:34) -> Hotel Transsilvanien 2 (OV) (19:30-21:20)',
 u'Alles steht Kopf (OV) (15:00-16:55) -> Crimson Peak (OV) (17:20-19:39) -> Der Marsianer - Rettet Mark Watney (OV) (19:30-22:14)',
 u'Alles steht Kopf (OV) (15:00-16:55) -> Crimson Peak (OV) (17:2

In [225]:
kino_program

,end_times,start_times,movie_name
0,1900-01-01 15:54:00,1900-01-01 13:30:00,the_martian
8,1900-01-01 16:00:00,1900-01-01 13:50:00,maze_runner
6,1900-01-01 15:29:00,1900-01-01 14:00:00,hotel_transylvania
15,1900-01-01 15:55:00,1900-01-01 14:20:00,alles_steht_kopf
11,1900-01-01 16:31:00,1900-01-01 14:40:00,pan
7,1900-01-01 18:09:00,1900-01-01 16:40:00,hotel_transylvania
1,1900-01-01 19:14:00,1900-01-01 16:50:00,the_martian
9,1900-01-01 19:01:00,1900-01-01 17:00:00,the_intern
16,1900-01-01 18:35:00,1900-01-01 17:00:00,alles_steht_kopf
2,1900-01-01 21:54:00,1900-01-01 19:30:00,the_martian


In [4]:
results

{u'count': 10,
 u'name': u'Cinestar OV Movies',
 u'results': {u'Movies': [{u'runtime': 144,
    u'showtimes': [u'13:30', u'16:50', u'19:30'],
    u'title': u'Der Marsianer - Rettet Mark Watney (OV)'},
   {u'runtime': 123, u'showtimes': u'20:15', u'title': u'Black Mass (OV)'},
   {u'runtime': 96, u'showtimes': u'20:15', u'title': u'American Ultra (OV)'},
   {u'runtime': 122, u'showtimes': u'20:00', u'title': u'Sicario (OV)'},
   {u'runtime': 95,
    u'showtimes': [u'14:20', u'15:00', u'17:00', u'17:30', u'20:15'],
    u'title': u'Alles steht Kopf (OV)'},
   {u'runtime': 111, u'showtimes': u'14:40', u'title': u'Pan (OV)'},
   {u'runtime': 119,
    u'showtimes': [u'17:20', u'20:10'],
    u'title': u'Crimson Peak (OV)'},
   {u'runtime': 103,
    u'showtimes': u'14:30',
    u'title': u'The Program - Um jeden Preis (OV)'},
   {u'runtime': 104,
    u'showtimes': [u'17:00', u'19:40'],
    u'title': u'Picknick mit B\xe4ren (OV)'},
   {u'runtime': 90,
    u'showtimes': [u'14:00', u'14:50', u'16:

In [153]:
"""
a -> c -> b
|    \
e     d

V = [a, b, c, d, e]
V.pop -> [a]
a.neighbors -> [c]
-- for each neighbor of a ... --

c.neighbors -> [b, d]
b.neighbors -> []
return b
d.neighbors -> []
return d
pre-pend c to returns -> [[c,b], [c,d]]

e.neighbors -> []
return e

pre-pend a to returns [[a,c,b],[a,c,d]]

def f(v):
    if neighbors.empty:
    return v
    else:
    return [v.prepend(f(x)) for x in v.neighbors]


start at a node
gather neighbors in a stack
pop neighbor in visited

"""

'\na -> c -> b\n|    e     d\n\nV = [a, b, c, d, e]\nV.pop -> [a]\na.neighbors -> [c]\n-- for each neighbor of a ... --\n\nc.neighbors -> [b, d]\nb.neighbors -> []\nreturn b\nd.neighbors -> []\nreturn d\npre-pend c to returns -> [[c,b], [c,d]]\n\ne.neighbors -> []\nreturn e\n\npre-pend a to returns [[a,c,b],[a,c,d]]\n\ndef f(v):\n    if neighbors.empty:\n    return v\n    else:\n    return [v.prepend(f(x)) for x in v.neighbors]\n\n\nstart at a node\ngather neighbors in a stack\npop neighbor in visited\n\n'

In [3]:
# hit kimono-made Cinestar showtimes API
results = json.load(urllib.urlopen('http://www.kimonolabs.com/api/ondemand/9q0fwoh2?kimmodify=1'))

In [4]:
results

{u'count': 10,
 u'name': u'Cinestar OV Movies',
 u'results': {u'Movies': [{u'runtime': 144,
    u'showtimes': [u'13:30', u'16:50', u'19:30'],
    u'title': u'Der Marsianer - Rettet Mark Watney (OV)'},
   {u'runtime': 123, u'showtimes': u'20:15', u'title': u'Black Mass (OV)'},
   {u'runtime': 96, u'showtimes': u'20:15', u'title': u'American Ultra (OV)'},
   {u'runtime': 122, u'showtimes': u'20:00', u'title': u'Sicario (OV)'},
   {u'runtime': 95,
    u'showtimes': [u'14:20', u'15:00', u'17:00', u'17:30', u'20:15'],
    u'title': u'Alles steht Kopf (OV)'},
   {u'runtime': 111, u'showtimes': u'14:40', u'title': u'Pan (OV)'},
   {u'runtime': 119,
    u'showtimes': [u'17:20', u'20:10'],
    u'title': u'Crimson Peak (OV)'},
   {u'runtime': 103,
    u'showtimes': u'14:30',
    u'title': u'The Program - Um jeden Preis (OV)'},
   {u'runtime': 104,
    u'showtimes': [u'17:00', u'19:40'],
    u'title': u'Picknick mit B\xe4ren (OV)'},
   {u'runtime': 90,
    u'showtimes': [u'14:00', u'14:50', u'16:

In [118]:
def initialize_movie_df(title, showtimes, runtime, ads_mins=0, trailers_mins=0):
    ads_timedelta = datetime.timedelta(minutes=ads_mins)
    trailers_timedelta = datetime.timedelta(minutes=trailers_mins)
    runtime_timedelta = datetime.timedelta(minutes=int(runtime))
    
    # if showtimes is a single string, list-comprehension will
    # move over the characters in the string, which is no-bueno.
    # Instead, wrap the string into a list
    if not isinstance(showtimes, list):
        showtimes = [showtimes]
    
    presentation_start_timedates = [datetime.datetime.strptime(showtime, '%H:%M') for showtime in showtimes]
    movie_start_timedates =  [start_time + ads_timedelta + trailers_timedelta for start_time in presentation_start_timedates]
    presentation_end_timedates = [movie_start_timedate + runtime_timedelta for movie_start_timedate in movie_start_timedates]
    output = pd.DataFrame({'start_time':presentation_start_timedates, \
                           'movie_start_time':movie_start_timedates,\
                           'movie_end_time':presentation_end_timedates})
    output['title'] = title
    return(output)

def make_showtimes(showtimes_api_results):
    # cobble together "kino_program", a DataFrame which has the start and end times for all
    # movies being pulled down from showtimes API
    kino_program = pd.DataFrame()

    for movie in showtimes_api_results['results']['Movies']:
        kino_program = \
        kino_program.append(ignore_index=True,\
                            other=initialize_movie_df(movie['title'], movie['showtimes'], movie['runtime'], 10, 10))

    return kino_program

In [119]:
kino_program = make_showtimes(results)
kino_program



,movie_end_time,movie_start_time,start_time,title
0,1900-01-01 16:14:00,1900-01-01 13:50:00,1900-01-01 13:30:00,Der Marsianer - Rettet Mark Watney (OV)
1,1900-01-01 19:34:00,1900-01-01 17:10:00,1900-01-01 16:50:00,Der Marsianer - Rettet Mark Watney (OV)
2,1900-01-01 22:14:00,1900-01-01 19:50:00,1900-01-01 19:30:00,Der Marsianer - Rettet Mark Watney (OV)
3,1900-01-01 22:38:00,1900-01-01 20:35:00,1900-01-01 20:15:00,Black Mass (OV)
4,1900-01-01 22:11:00,1900-01-01 20:35:00,1900-01-01 20:15:00,American Ultra (OV)
5,1900-01-01 22:22:00,1900-01-01 20:20:00,1900-01-01 20:00:00,Sicario (OV)
6,1900-01-01 16:15:00,1900-01-01 14:40:00,1900-01-01 14:20:00,Alles steht Kopf (OV)
7,1900-01-01 16:55:00,1900-01-01 15:20:00,1900-01-01 15:00:00,Alles steht Kopf (OV)
8,1900-01-01 18:55:00,1900-01-01 17:20:00,1900-01-01 17:00:00,Alles steht Kopf (OV)
9,1900-01-01 19:25:00,1900-01-01 17:50:00,1900-01-01 17:30:00,Alles steht Kopf (OV)
